In [1]:
!pip install -q tflite-model-maker
!pip install -q pycocotools

     |████████████████████████████████| 616 kB 4.3 MB/s 
     |████████████████████████████████| 77 kB 4.4 MB/s 
     |████████████████████████████████| 3.4 MB 39.8 MB/s 
     |████████████████████████████████| 1.1 MB 36.3 MB/s 
     |████████████████████████████████| 87 kB 5.8 MB/s 
     |████████████████████████████████| 120 kB 47.5 MB/s 
     |████████████████████████████████| 1.2 MB 31.6 MB/s 
     |████████████████████████████████| 1.1 MB 40.6 MB/s 
     |████████████████████████████████| 596 kB 44.9 MB/s 
     |████████████████████████████████| 6.4 MB 36.5 MB/s 
     |████████████████████████████████| 213 kB 53.7 MB/s 
     |████████████████████████████████| 840 kB 23.9 MB/s 
     |████████████████████████████████| 25.3 MB 1.6 MB/s 
     |████████████████████████████████| 352 kB 47.0 MB/s 
     |████████████████████████████████| 99 kB 8.6 MB/s 
     |████████████████████████████████| 47.7 MB 66 kB/s 
     |████████████████████████████████| 211 kB 45.5 MB/s 


In [2]:
from tensorflow_examples.lite.model_maker.core.data_util.image_dataloader import ImageClassifierDataLoader
from tensorflow_examples.lite.model_maker.core.task import image_classifier

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import tflite_model_maker.image_classifier 

In [2]:
!pip install -q git+https://github.com/tensorflow/examples.git#egg=tensorflow-examples[model_maker]

    Running setup.py install for tensorflow-examples ... done
  DEPRECATION: tensorflow-examples was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


In [4]:
import os 
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

In [5]:
!kaggle datasets download -d rahilmehtaucoe2784/cloth-pattern

 99% 1.69G/1.70G [00:56<00:00, 32.6MB/s]
100% 1.70G/1.70G [00:56<00:00, 32.1MB/s]


In [6]:
%%capture
!unzip \*.zip && rm *.zip &> /dev/null

In [9]:
image_path = "/content/NEW LARGE IMAGE "
data = ImageClassifierDataLoader.from_folder(image_path)
train_data, rest_data = data.split(0.8)
validation_data = rest_data

INFO:tensorflow:Load image with size: 4710, num_label: 7, labels: checkered, dots, florals , horizontal strip, paisley pattern, vertical strip, zig zag .


INFO:tensorflow:Load image with size: 4710, num_label: 7, labels: checkered, dots, florals , horizontal strip, paisley pattern, vertical strip, zig zag .


In [28]:
model = image_classifier.create(train_data, 
                                model_spec= tflite_model_maker.image_classifier.EfficientNetLite0Spec(), 
                                epochs=5, 
                                #validation_data=validation_data
                                )

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2_2 (Hub  (None, 1280)             3413024   
 KerasLayerV1V2)                                                 
                                                                 
 dropout_2 (Dropout)         (None, 1280)              0         
                                                                 
 dense_2 (Dense)             (None, 7)                 8967      
                                                                 
Total params: 3,421,991
Trainable params: 8,967
Non-trainable params: 3,413,024
_________________________________________________________________
None
Epoch 1/5


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


115/117 [============================>.] - ETA: 3s - loss: 0.9642 - accuracy: 0.7807

InvalidArgumentError: ignored

In [27]:
from struct import unpack
from tqdm import tqdm
import os


marker_mapping = {
    0xffd8: "Start of Image",
    0xffe0: "Application Default Header",
    0xffdb: "Quantization Table",
    0xffc0: "Start of Frame",
    0xffc4: "Define Huffman Table",
    0xffda: "Start of Scan",
    0xffd9: "End of Image"
}


class JPEG:
    def __init__(self, image_file):
        with open(image_file, 'rb') as f:
            self.img_data = f.read()
    
    def decode(self):
        data = self.img_data
        while(True):
            marker, = unpack(">H", data[0:2])
            # print(marker_mapping.get(marker))
            if marker == 0xffd8:
                data = data[2:]
            elif marker == 0xffd9:
                return
            elif marker == 0xffda:
                data = data[-2:]
            else:
                lenchunk, = unpack(">H", data[2:4])
                data = data[2+lenchunk:]            
            if len(data)==0:
               raise TypeError("issue reading jpeg file")           


bads = []

image_path = '/content/NEW LARGE IMAGE /zig zag '
for dirName, subdirList, fileList in os.walk(image_path):
    imagesList = fileList
    for img in tqdm(imagesList):
      image = os.path.join(image_path,img)
      image = JPEG(image) 
    try:
      image.decode()   
    except:
      bads.append(img)

print(bads)
for name in bads:
  os.remove(osp.join(root_img,name))


100%|██████████| 560/560 [00:00<00:00, 3886.64it/s]

[]
